In [2]:
import os, sys
import random
import numpy as np

In [3]:
from PySide2.QtCore import QUrl, Qt, QCoreApplication
from PySide2.QtWidgets import QApplication
from PySide2.QtQml import QQmlApplicationEngine

from PySide2.QtCore import QObject, Signal, Slot, Property
from PySide2.QtCore import QAbstractListModel, QAbstractTableModel, QModelIndex, QByteArray

In [4]:
import cryspy

# Python Proxy Model

In [33]:
class PythonModel(QObject):

    def __init__(self, parent=None):
        super().__init__(parent)
        self._data_dict = {}
        self._fitables_list = []
        self._atom_site_list = []
        self._cell_dict = {}

    # ##############################################
    # Private methods
    # ##############################################

    def _get_data_dict(self):
        return self._data_dict

    def _set_data_dict(self, data):
        #self._data_dict = data.toVariant() # don't preserve order
        for key, value in data.toVariant().items():
            for subkey, subvalue in value.items():
                self._data_dict[key][subkey] = subvalue
        self.dataChanged.emit()

    def _get_fitables_list(self):
        self._fitables_list.clear()
        # ADD header now
        for key, value in self._data_dict.items():
            for subkey, subvalue in value.items():
                try:
                    self._fitables_list.append({
                        'datablock': key,
                        'group': '',
                        'subgroup': '',
                        'name': subkey,
                        'value': subvalue['value'],
                        'error': subvalue['error'],
                        'refine': subvalue['refine']
                        })
                except:
                    pass
                try:
                    for subsubkey, subsubvalue in subvalue.items():
                        for subsubsubkey, subsubsubvalue in subsubvalue.items():
                            self._fitables_list.append({
                                'datablock': key,
                                'group': subkey,
                                'subgroup': subsubkey,
                                'name': subsubsubkey,
                                'value': subsubsubvalue['value'],
                                'error': subsubsubvalue['error'],
                                'refine': subsubsubvalue['refine']
                                })
                except:
                    pass
        return self._fitables_list

    def _get_atom_site_list(self):
        self._atom_site_list.clear()
        for key, value in self._data_dict.items():
            try:
                for subkey, subvalue in value['atom_site'].items():
                   self._atom_site_list.append({'datablock': key, 'label': subkey, **subvalue})
            except:
                pass
        return self._atom_site_list

    def _get_cell_dict(self):
        for key, value in self._data_dict.items():
            try:
                self._cell_dict['cell_length_a'] = value['cell_length_a']
                self._cell_dict['cell_length_b'] = value['cell_length_b']
                self._cell_dict['cell_length_c'] = value['cell_length_c']
                self._cell_dict['cell_angle_alpha'] = value['cell_angle_alpha']
                self._cell_dict['cell_angle_beta'] = value['cell_angle_beta']
                self._cell_dict['cell_angle_gamma'] = value['cell_angle_gamma']
                self._cell_dict['datablock'] = key
            except:
                pass
        return self._cell_dict

    def _set_fitable_from_rhochi_model(self, obj, name, title, datablock, group="", subgroup=""):
        if datablock not in self._data_dict.keys():
            self._data_dict[datablock] = {}
        if group and subgroup:
            pass
        else:
            if name not in self._data_dict[datablock].keys():
                self._data_dict[datablock][name] = {}
            if isinstance(obj, float) or isinstance(obj, int):
                self._data_dict[datablock][name]['value'] = '{:.4f}'.format(obj)
                self._data_dict[datablock][name]['error'] = ''
                self._data_dict[datablock][name]['refine'] = False
                self._data_dict[datablock][name]['title'] = title
            elif isinstance(obj, neupy.f_common.cl_variable.Variable):
                self._data_dict[datablock][name]['value'] = '{:.4f}'.format(obj.__getitem__(0))
                self._data_dict[datablock][name]['error'] = '{:.4f}'.format(obj.__getitem__(4))
                self._data_dict[datablock][name]['refine'] = obj.__getitem__(1)
                self._data_dict[datablock][name]['title'] = title
            else:
                print("Unknown object type", type(obj))

    def _set_data_dict_from_rhochi_model(self):
        #print("_set_data_dict_from_rhochi_model")
        self._data_dict.clear()
        for phase in self._model._list_crystal:
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('a'), 'cell_length_a', 'a (\u212B)', phase.get_val('name'))
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('b'), 'cell_length_b', 'b (\u212B)', phase.get_val('name'))
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('c'), 'cell_length_c', 'c (\u212B)', phase.get_val('name'))
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('alpha'), 'cell_angle_alpha', 'alpha (\u212B)', phase.get_val('name'))
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('beta'), 'cell_angle_beta', 'beta (\u212B)', phase.get_val('name'))
            self._set_fitable_from_rhochi_model(phase.get_val('cell').get_val('gamma'), 'cell_angle_gamma', 'gamma (\u212B)', phase.get_val('name'))

    def _update_rhochi_cell_model(self):
        #print("_update_rhochi_cell_model")
        for phase in self._model._list_crystal:
            a = float(self._data_dict[phase.get_val('name')]['cell_length_a']['value'])
            b = float(self._data_dict[phase.get_val('name')]['cell_length_b']['value'])
            c = float(self._data_dict[phase.get_val('name')]['cell_length_c']['value'])
            alpha = float(self._data_dict[phase.get_val('name')]['cell_angle_alpha']['value'])
            beta = float(self._data_dict[phase.get_val('name')]['cell_angle_beta']['value'])
            gamma = float(self._data_dict[phase.get_val('name')]['cell_angle_gamma']['value'])
            #
            if self._data_dict[phase.get_val('name')]['cell_length_a']['refine']:
                a = neupy.f_common.cl_variable.Variable(val = a, ref=True)
            if self._data_dict[phase.get_val('name')]['cell_length_b']['refine']:
                b = neupy.f_common.cl_variable.Variable(val = b, ref=True)
            if self._data_dict[phase.get_val('name')]['cell_length_c']['refine']:
                c = neupy.f_common.cl_variable.Variable(val = c, ref=True)
            if self._data_dict[phase.get_val('name')]['cell_angle_alpha']['refine']:
                alpha = neupy.f_common.cl_variable.Variable(val = alpha, ref=True)
            if self._data_dict[phase.get_val('name')]['cell_angle_beta']['refine']:
                beta = neupy.f_common.cl_variable.Variable(val = beta, ref=True)
            if self._data_dict[phase.get_val('name')]['cell_angle_gamma']['refine']:
                gamma = neupy.f_common.cl_variable.Variable(val = gamma, ref=True)
            #
            cell = phase.get_val('cell')
            cell.set_val(a = a, b = b, c = c, alpha = alpha, beta = beta, gamma = gamma)
            #
            phase.set_val(cell = cell)

    # ##############################################
    # Signals
    # ##############################################

    #@Signal
    #def dataChanged(self):
    #    pass

    dataChanged = Signal()

    # ##############################################
    # QML accessible public slots
    # ##############################################

    #@Slot('QJSValue')
    #def set_data(self, data):
    #    self._data = data.toVariant()
    #    self.dataChanged.emit()

    @Slot()
    def random_change_cell_length_a(self):
        phase = self._model._list_crystal[0]
        self._data_dict[phase.get_val('name')]['cell_length_a']['value'] = '{:.4f}'.format(random.random())
        self.dataChanged.emit()

    @Slot(str)
    def load_rhochi_model_and_update_proxy(self, path):
        #print("load_rhochi_model_and_update_proxy: ", path)
        self._rcif_file_absolute_path = path
        self._rcif = neupy.f_rcif.RCif()
        self._rcif.load_from_file(self._rcif_file_absolute_path)
        self._model = neupy.f_api_rcif.api_rcif_model.conv_rcif_to_model(self._rcif)
        self._set_data_dict_from_rhochi_model()
        self._project_opened = True
        self.dataChanged.emit()

    @Slot(result='QVariant')
    def refine_model(self):
        #print("refine_model")
        self.update_rhochi_model()
        try:
            res = self._model.refine_model()
            if res is None:
                return "No refinable parameters found"
        except np.linalg.LinAlgError as err:
            return str(err)
        except:
            return "Unknow error during refinement"
        self._set_data_dict_from_rhochi_model()
        self.dataChanged.emit()
        return dict(res)

    @Slot()
    def update_rhochi_model(self):
        #print("update_rhochi_model")
        self._update_rhochi_cell_model()

    # ##############################################
    # QML accessible properties
    # ##############################################

    data = Property('QVariant', _get_data_dict, _set_data_dict, notify=dataChanged)
    fitables_list = Property('QVariant', _get_fitables_list, notify=dataChanged)
    atom_site_list = Property('QVariant', _get_atom_site_list, notify=dataChanged)
    cell_dict = Property('QVariant', _get_cell_dict, notify=dataChanged)

# Path to RCif

In [5]:
cryspy_ipynb_path = %pwd
main_rcif_path = os.path.join(cryspy_ipynb_path, "cryspy-master", "examples", "Fe3O4_0T_powder_1d", "main.rcif")

In [6]:
rhochi = cryspy.rhochi_read_file(main_rcif_path)

In [7]:
for datablock in rhochi.crystals:
    print(datablock.label)
    print(datablock.cell.a)
    print(datablock.cell.a.name, datablock.cell.a.value, datablock.cell.a.sigma, datablock.cell.a.refinement)
    print(datablock.cell.alpha)

Fe3O4
8.56212 (name: _cell_length_a)
_cell_length_a 8.56212 0.0 False
90.0 (name: _cell_angle_alpha)


In [8]:
rhochi.crystals[0]

Crystal:
label: Fe3O4

Cell: 
 a: 8.56212 (name: _cell_length_a)
 b: 8.56212 (name: _cell_length_b)
 c: 8.56212 (name: _cell_length_c)
 alpha: 90.0 (name: _cell_angle_alpha)
 beta: 90.0 (name: _cell_angle_beta)
 gamma: 90.0 (name: _cell_angle_gamma)
 bravais_lattice: cubic
 volume: 627.688
 B matrix is:
   0.11679   0.00000   0.00000
   0.00000   0.11679  -0.00000
   0.00000   0.00000   0.11679
 inversed B matrix is:
   8.56212  -0.00000  -0.00000
   0.00000   8.56212   0.00000
   0.00000   0.00000   8.56212

SpaceGroup:
 given name: Fd-3m
 choice: 2
 name: Fd-3m
 number: 227

AtomSite:
 label type_symbol x        y        z        adp_type b_iso    occupancy
 Fe3A     Fe3+     0.125    0.125    0.125    uani     0.0      1.0     
 Fe3B     Fe3+     0.5      0.5      0.5      uani     0.0      1.0     
 O1       O2-      0.25521  0.25521  0.25521  uiso     0.0      1.0     

 for type_symbol 'O2-'
 scat_length_neutron: 0.5803 cm**-12
 j0_A:  0.9990,  j0_a: 12.0965,  j0_B:  0.2885,  j0_

In [13]:
rhochi.crystals[0].atom_site

AtomSite:
 label type_symbol x        y        z        adp_type b_iso    occupancy
 Fe3A     Fe3+     0.125    0.125    0.125    uani     0.0      1.0     
 Fe3B     Fe3+     0.5      0.5      0.5      uani     0.0      1.0     
 O1       O2-      0.25521  0.25521  0.25521  uiso     0.0      1.0     

 for type_symbol 'O2-'
 scat_length_neutron: 0.5803 cm**-12
 j0_A:  0.9990,  j0_a: 12.0965,  j0_B:  0.2885,  j0_b:  0.1291
 j0_C:  0.1143,  j0_c: -0.2297,  j0_D: -0.4068
 j2_A:  0.0000,  j2_a:  0.0000,  j2_B:  0.0000,  j2_b:  0.0000
 j2_C:  0.0000,  j2_c:  0.0000,  j2_D:  0.0000

 for type_symbol 'Fe3+'
 scat_length_neutron: 0.9450 cm**-12
 j0_A:  0.3972,  j0_a: 13.2442,  j0_B:  0.6295,  j0_b:  4.9034
 j0_C: -0.0314,  j0_c:  0.3496,  j0_D:  0.0044
 j2_A:  1.3602,  j2_a: 11.9976,  j2_B:  1.5188,  j2_b:  5.0025
 j2_C:  0.4705,  j2_c:  1.9914,  j2_D:  0.0038

In [18]:
rhochi.crystals[0].atom_site.label

('Fe3A', 'Fe3B', 'O1')

In [17]:
rhochi.crystals[0].atom_site.x

(0.125 (name: _atom_site_fract_x),
 0.5 (name: _atom_site_fract_x),
 0.25521 (name: _atom_site_fract_x))

In [53]:
for datablock in rhochi.experiments:
    print(datablock.meas)
    print(type(datablock.meas))




PdMeas:
 ttheta           up     up_sigma         down   down_sigma
   4.00    465.80000    128.97000    301.88000    129.30000
   4.20    323.78000    118.22000    206.06000    120.00000
   4.40    307.14000    115.90000    230.47000    116.53000
   4.60    172.13000    115.84000    236.52000    113.74000
   4.80    164.31000    110.75000    291.60000    110.73000
   5.00    166.47000    106.51000    426.73000    109.08000
   5.20    217.67000    104.66000    183.24000    107.79000
   5.40     84.01000     99.91000    179.87000     99.98000
   5.60    220.87000     98.72000    297.76000     99.83000
   5.80    223.46000     98.77000    149.78000     98.18000
   6.00    408.25000     96.89000    356.86000     96.41000
   6.20    381.21000     94.03000    287.26000     96.94000
   6.40    239.85000     94.06000    101.64000     95.30000
   6.60     58.16000     90.93000    194.35000     89.90000
   6.80    127.79000     88.33000    273.48000     89.35000
   7.00    148.69000     87.5000

In [16]:
for fitable in rhochi.get_variables():
    print(fitable)

-3.468() (name: _atom_site_magnetism_aniso_chi_11, refinement: True)


In [23]:
print(rhochi.crystals[0].cell.a)

8.56212 (name: _cell_length_a)


# Use NeuPy Model

In [7]:
rcif = neupy.f_rcif.RCif()
rcif.load_from_file(rcif_file_path)
model = neupy.f_api_rcif.api_rcif_model.conv_rcif_to_model(rcif)

In [8]:
for phase in model._list_crystal:
    print(phase.get_val('cell').get_val('a'))

8.56212


In [23]:
for phase in model._list_experiment:
    print(phase.get_val('name'))

pnd


In [13]:
for phase in model._list_experiment:
    print(phase.get_val('observed_data').get_val('tth'))

[ 4.   4.2  4.4  4.6  4.8  5.   5.2  5.4  5.6  5.8  6.   6.2  6.4  6.6
  6.8  7.   7.2  7.4  7.6  7.8  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4
  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2
 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.
 15.2 15.4 15.6 15.8 16.  16.2 16.4 16.6 16.8 17.  17.2 17.4 17.6 17.8
 18.  18.2 18.4 18.6 18.8 19.  19.2 19.4 19.6 19.8 20.  20.2 20.4 20.6
 20.8 21.  21.2 21.4 21.6 21.8 22.  22.2 22.4 22.6 22.8 23.  23.2 23.4
 23.6 23.8 24.  24.2 24.4 24.6 24.8 25.  25.2 25.4 25.6 25.8 26.  26.2
 26.4 26.6 26.8 27.  27.2 27.4 27.6 27.8 28.  28.2 28.4 28.6 28.8 29.
 29.2 29.4 29.6 29.8 30.  30.2 30.4 30.6 30.8 31.  31.2 31.4 31.6 31.8
 32.  32.2 32.4 32.6 32.8 33.  33.2 33.4 33.6 33.8 34.  34.2 34.4 34.6
 34.8 35.  35.2 35.4 35.6 35.8 36.  36.2 36.4 36.6 36.8 37.  37.2 37.4
 37.6 37.8 38.  38.2 38.4 38.6 38.8 39.  39.2 39.4 39.6 39.8 40.  40.2
 40.4 40.6 40.8 41.  41.2 41.4 41.6 41.8 42.  42.2 42.4 42.6 42.8 43.
 43.2 43.

# Use Python Proxy Model 

In [37]:
pythonModel = PythonModel()
pythonModel.load_rhochi_model_and_update_proxy(rcif_file_path)

In [41]:
for key, value in pythonModel.cell_dict.items():
    print(key, value)

cell_length_a {'value': '8.5621', 'error': '', 'refine': False, 'title': 'a (Å)'}
cell_length_b {'value': '8.5621', 'error': '', 'refine': False, 'title': 'b (Å)'}
cell_length_c {'value': '8.5621', 'error': '', 'refine': False, 'title': 'c (Å)'}
cell_angle_alpha {'value': '90.0000', 'error': '', 'refine': False, 'title': 'alpha (Å)'}
cell_angle_beta {'value': '90.0000', 'error': '', 'refine': False, 'title': 'beta (Å)'}
cell_angle_gamma {'value': '90.0000', 'error': '', 'refine': False, 'title': 'gamma (Å)'}
datablock Fe3O4


# Tests

In [18]:
data = []
m = 3
data.append([i for i in range(m)])
data.append([random.randrange(1000) for i in range(m)])
data.append([0.1 for i in range(m)])
data.append([random.randrange(1000) for i in range(m)])

In [19]:
print(data)

[[0, 1, 2], [164, 504, 18], [0.1, 0.1, 0.1], [869, 740, 52]]


In [20]:
print(data[0])

[0, 1, 2]


In [21]:
len(data)

4

In [22]:
len(data[0])

3

In [26]:
P = [[1, 2, 3], [4, 5, 6]]

In [28]:
P

[[1, 2, 3], [4, 5, 6]]

In [29]:
P[0] = [7, 8, 9]

In [30]:
P

[[7, 8, 9], [4, 5, 6]]